## RAG-PDF

Este cuaderno procesa documentos PDF, extrayendo y dividiendo su contenido en fragmentos de texto. Luego, los almacena en una base de datos de vectores para facilitar la búsqueda. Utilizando el modelo **Llama 3.2**, el sistema responde a preguntas sobre los PDFs, generando respuestas precisas basadas en la información almacenada.

Importa las librerías necesarias

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

Carga los pdfs, extraemos los textos y los dividimos

In [3]:
pdf_files = ["documents/pruebaLLM.pdf","documents/pruebaLLM2.pdf", "documents/MartinScorsese.pdf"]  

all_documents = []

for document in pdf_files:
    loader = PyPDFLoader(document) # Carga el documento
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)    # Divide el texto en chunks
    documents = text_splitter.split_documents(data)
    all_documents.extend(documents)

print(f"Se cargaron y dividieron {len(all_documents)} fragmentos de texto.")

Se cargaron y dividieron 220 fragmentos de texto.


Configura un modelo de Hugging Face para generar embeddings de texto.

In [4]:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/martin/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Crear un vector store de Chroma usando los documentos y el modelo de embeddings


In [5]:
vector_store = Chroma.from_documents(
    documents=all_documents,
    collection_name="pdfs_2",
    embedding=embeddings,  # Pasar el modelo de embeddings aquí
)

print(f"Se creó el vector store con {len(all_documents)} documentos.")

Se creó el vector store con 220 documentos.


Utiliza una función de embeddings para representar los datos

In [6]:
vector_store = Chroma(
    collection_name="pdfs_2",
    embedding_function=embeddings,
)

Convierte el vector store en un retriever  para realizar consultas basadas en similitud de texto

In [7]:
retriever = vector_store.as_retriever()

Pregunta de ejemplo

In [ ]:
question = "Como es la filmografia de Martin Scorsese"
docs = vector_store.similarity_search(question, k=10)
len(docs)

10

Vemos los textos más relacionados a la pregunta

In [9]:
docs

[Document(metadata={'page': 21, 'source': 'documents/MartinScorsese.pdf'}, page_content='500. Paris, 1996.\nMarino, Giuseppe Cario. Historiade laMafia. Barcelona: Ediciones B, 2002.\nMillas, Jorge. LaViolenciay sus Máscaras. Santiago: Aconcagua, 1976.\nRuiz, Raúl. PoéticaDel Cine. Santiago: Sudamericana, 2000.\nScorsese, Martínyotros. APersonalJourneyWithMartínScorseseThroughAmericanMovie.\nLondon: Akal, 1997.\nScorsese, Martínyotros. Conversaciones con Scorsese. Madrid: Plot, 1987.'),
 Document(metadata={'page': 20, 'source': 'documents/MartinScorsese.pdf'}, page_content='Michael, amenazando con delirantes castigos\ncontraJohnyBoy, antelamiradacomplacientey\nlaenigmáticasonrisade Shorty. Sibienes cierto,\nparael estrenodeMeanStreets, Scorsesenoera\nun cineasta conocido, su presencia como actor\ndebería pasar inadvertida. Hoy, dada su fama\ny manejo mediático, la solución que entrega\nno deja de ser sorprendente. La filmación de'),
 Document(metadata={'page': 8, 'source': 'documents/Ma

Crea una cadena de procesamiento de lenguaje natural utilizando un LLM local (Ollama) para responder preguntas basadas en un contexto dado

In [10]:
# Prompt: Se define una plantilla de pregunta-respuesta que toma un contexto y una pregunta como entrada
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM: Se configura el modelo llama3.2 para generar respuestas de manera local
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain Se construye un pipeline que pasa el contexto desde el retriever
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

Probamos el modelo

In [11]:
chain.invoke("pelicula y color favorito de Juan(pruebaLLM2.pdf)")

'La respuesta a la pregunta es:\n\n- Película favorita: Mulholland Drive\n- Color favorito: Amarillo'

In [12]:
chain.invoke("Quien es Martin Scorsese")

'Según el texto proporcionado, Martin Scorsese es un cineasta conocido. Se menciona que su presencia como actor debería pasar inadvertida debido a su fama y manejo mediático, lo que sugiere que no es un actor destacado por su actuación en películas. Sin embargo, su trabajo como director de cine es el que se destaca en el texto.'

In [13]:
chain.invoke("Que relación tiene la violencia es sus películas?")

'La violencia es un tema central en las películas de Martin Scorsese, según el texto proporcionado. Se menciona que la violencia es una forma de vida en muchas de sus obras, incluyendo "Mean Streets", y que se presenta como un aspecto inherente al hombre y a la sociedad. También se menciona que la violencia es una constante opresión del sentido aparentoso de la realidad en su cine, especialmente en películas como "Casino" (1995). Además, se destaca que la violencia en Scorsese no es solo física, sino también psicológica y emocional.\n\nEn general, parece que la violencia en las películas de Martin Scorsese es un tema complejo y multifacético que aborda temas como la tragedia, el drama y la exploración del ser humano.'

In [14]:
chain.invoke("Que hay en el documento sobre Goodfellas")

'En el documento, hay varias secciones que tratan sobre Goodfellas:\n\n1. La primera sección (página 18) menciona la evocación de la "ostentosa elegancia del gángster" en el cine de género y cómo la película Goodfellas convoca a un grupo de personajes, incluyendo a Michael, en un mundo determinado por ambigüedades.\n2. La segunda sección (páginas 12) habla sobre las enseñanzas de Paul Cicero, el padrino de Goodfellas, y su autodestrucción, así como la moral gangsteril presente en el film y su relación con la tradición del cine de género.\n3. La tercera sección (página 4) menciona la importancia de seguir la trayectoria criminal de un padre en la familia Corleone y cómo esta familia es una organización dentro del Estado, que ha convertido el crimen en un modo de vida.\n4. La última sección (página 12) analiza la estructura del relato y la eficacia de Scorsese en la planificación y montaje del film, destacando la circularidad entre las cuatro escenas.\n\nEn resumen, el documento ofrece u

In [15]:
chain.invoke("Resume el documento 'LA VIOLENCIA COMO OPCIÓN CREATIVA EN LA OBRA DE MARTÍN SCORSESE'")

'El documento "La violencia como opción creativa en la obra de Martin Scorsese" parece ser un análisis filosófico y cinematográfico sobre la figura del director Martin Scorsese. A continuación, se presentan los puntos clave resumidos del texto:\n\n1. **Violencia como opción creativa**: El documento establece que la violencia es una fuerza fundamental en la obra de Scorsese, que utiliza para expresar su visión y crear un mundo cinematográfico único.\n2. **Análisis de algunas películas**: A través de análisis de algunas de sus películas, como "Mean Streets" (1973) y "El Padrino" (1972), se demuestra cómo Scorsese utiliza la violencia para explorar temas como la fe, el fatalismo y la moralidad.\n3. **La poesía de la violencia**: El texto afirma que la violencia en Scorsese es una forma de poesía, una forma de expresión que se inscribe en un lenguaje visual y narrativo único.\n4. **Consecuencia moral a la violencia**: Aunque admite que la violencia es una fuerza destructiva, el documento s

In [16]:
chain.invoke("Que relación tiene la mafia en sus películas?")

'La mafia es un tema central en las películas de Martin Scorsese, y su relación con él se puede resumir en que la organización criminal se presenta como una parte integral del Estado, más allá de su naturaleza ilegal. Según la cita incluida en el texto, Scorsese afirma que "la mafia es la más fuerte" y que "Primero está la familia, es decir, la tuya más la mafia, luego la iglesia". Esto sugiere que la mafia no solo es una estructura social pervertida, sino también un elemento fundamental de la sociedad italiana.\n\nAdemás, la película de Scorsese parece explorar la complejidad de la mafia, mostrando su lado oscuro y brutal, pero también su faceta más humana y protectora. La cita del número 78 menciona que los gángsters pueden ser "manipuladores y muy brutos" al mismo tiempo, lo que sugiere una dimensión psicológica de la mafia.\n\nEn general, la relación de la mafia con las películas de Scorsese se centra en su representación como una parte integral de la sociedad italiana, con sus pro

En conclusión, el modelo responde de manera efectiva a las preguntas sobre los PDFs, proporcionando respuestas relevantes y coherentes basadas en el contexto extraído de los documentos. La integración del recuperador de vectores y el modelo LLM local permite una comprensión precisa de las consultas, demostrando su capacidad para manejar información textual de manera eficiente.